# About
* **Author**: Adil Rashitov
* **Created at**: 17.08.2021
* **Goal**: Solve final assignment

![pictures](pictures/FINAL_TASK_2.png)

In [1]:
# Imports / Configs / Global vars

# Import of native python tools
import os
import json
from functools import reduce

# Import of base ML stack libs
import numpy as np
import sklearn as sc

# Multiprocessing for Mac / Linux
import platform
platform.system()
if platform.system() == 'Darwin':
    from multiprocess import Pool
else:
    from multiprocessing import Pool

# Visualization libraries
import plotly.express as px

# Logging configuraiton
import logging
logging.basicConfig(format='[ %(asctime)s ][ %(levelname)s ]: %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Ipython configs
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
display(HTML("<style>.container { width:100% !important; }</style>"))
InteractiveShell.ast_node_interactivity = 'all'

# Pandas configs
import pandas as pd
import geopandas as gpd
pd.options.display.max_rows = 350
pd.options.display.max_columns = 250

# Jupyter configs
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

from ortools.linear_solver import pywraplp

# Data

In [2]:
def prepare_distance_matrix():
    data = """
        District
        1
        2
        3
        4
        5
        6
        7
        8
        1
        0
        3
        4
        6
        8
        9
        8
        10
        2
        3
        0
        5
        4
        8
        6
        12
        9
        3
        4
        5
        0
        2
        2
        3
        5
        7
        4
        6
        4
        2
        0
        3
        2
        5
        4
        5
        8
        8
        2
        3
        0
        2
        2
        4
        6
        9
        6
        3
        2
        2
        0
        3
        2
        7
        8
        12
        5
        5
        2
        3
        0
        2
        8
        10
        9
        7
        4
        4
        2
        2
        0
    """
    data = pd.Series(data.split('\n')[1:-1]).str.lstrip().str.rstrip()
    X = np.reshape(np.array(data), (9, 9))
    X = np.int16(X[1:, 1:])
    return X

D_matrix = prepare_distance_matrix()
MAX_DISTANCE = int(D_matrix.max())

In [3]:
def prepare_population_vec():
    data = """
        District
        Population
        1
        40
        2
        30
        3
        35
        4
        20
        5
        15
        6
        50
        7
        45
        8
        60
    """

    data = pd.Series(data.split('\n')[1:-1]).str.lstrip()
    X = np.reshape(np.array(data), (9, 2)).T[1][1:]
    X = pd.Series(X).astype(int).values
    return X

p_vec = prepare_population_vec()
MAX_POPULATION = int(np.max(p_vec))

# Main


In [4]:
m = 2
n = 8

### Step 1 : X variables definition

In [5]:
# Model.
from ortools.sat.python import cp_model
model = cp_model.CpModel()


# Change variable
x_vec = [model.NewIntVar(0, m, f"district_{x}") for x in range(D_matrix.shape[0])]

# Vector of presence ambulance in district
has_ambulance = []
for _id, x in enumerate(x_vec):
    bool_var = model.NewBoolVar(f"has_ambulance_{_id}")
    model.Add(bool_var == x > 0)
    has_ambulance.append(bool_var)

### Step 2 : Objective function formulation

In [6]:
# x_matrix = [[1, 0, 0, 0, 0, 0, 0, 1]] * D_matrix.shape[0]
x_matrix = [x_vec] * D_matrix.shape[0]

In [7]:
# Set Max distances to ones column not selected & i=j

D_amb_district_2 = []

D_amb_district_1 = D_matrix * x_matrix
for i in range(D_amb_district_1.shape[0]):

    distrcit_distance = []
    for j in range(D_amb_district_1.shape[1]):

        # 1. Registiring new variable
        d_i_j = model.NewIntVar(0, MAX_DISTANCE, f'min_amb_distr_{i}_{j}')

        _ = model.Add(d_i_j == D_amb_district_1[i][j]).OnlyEnforceIf(has_ambulance[j])
        _ = model.Add(d_i_j == MAX_DISTANCE).OnlyEnforceIf(has_ambulance[j].Not())        
        logging.info(f"Constrain is set for d_{i}_{j}")

        distrcit_distance.append(d_i_j)
    D_amb_district_2.append(distrcit_distance)

D_amb_district_2 = np.array(D_amb_district_2)

[ 09/17/2021 07:38:58 AM ][ INFO ]: Constrain is set for d_0_0
[ 09/17/2021 07:38:58 AM ][ INFO ]: Constrain is set for d_0_1
[ 09/17/2021 07:38:58 AM ][ INFO ]: Constrain is set for d_0_2
[ 09/17/2021 07:38:58 AM ][ INFO ]: Constrain is set for d_0_3
[ 09/17/2021 07:38:58 AM ][ INFO ]: Constrain is set for d_0_4
[ 09/17/2021 07:38:58 AM ][ INFO ]: Constrain is set for d_0_5
[ 09/17/2021 07:38:58 AM ][ INFO ]: Constrain is set for d_0_6
[ 09/17/2021 07:38:58 AM ][ INFO ]: Constrain is set for d_0_7
[ 09/17/2021 07:38:58 AM ][ INFO ]: Constrain is set for d_1_0
[ 09/17/2021 07:38:58 AM ][ INFO ]: Constrain is set for d_1_1
[ 09/17/2021 07:38:58 AM ][ INFO ]: Constrain is set for d_1_2
[ 09/17/2021 07:38:58 AM ][ INFO ]: Constrain is set for d_1_3
[ 09/17/2021 07:38:58 AM ][ INFO ]: Constrain is set for d_1_4
[ 09/17/2021 07:38:58 AM ][ INFO ]: Constrain is set for d_1_5
[ 09/17/2021 07:38:58 AM ][ INFO ]: Constrain is set for d_1_6
[ 09/17/2021 07:38:58 AM ][ INFO ]: Constrain is set fo

In [8]:
# Extraction min distances between ambulances to districts from matrix
min_distances = []


for _id, j in enumerate(range(D_amb_district_2.shape[0])):
    min_distance = model.NewIntVar(0, MAX_DISTANCE, f'min_distance_{_id}')
    _ = model.AddMinEquality(min_distance, D_amb_district_2[j])
    min_distances.append(min_distance)

min_distances = np.array(min_distances)

In [9]:
# Extraction of product min distances between ambulances to districts and population
prod_min_d_population = p_vec * min_distances

c_prod_min_d_population = []
for _id, product_d_pop in enumerate(prod_min_d_population):
    intermediate = model.NewIntVar(0, MAX_DISTANCE*MAX_POPULATION, f'product_pop_min_dist_{_id}')
    _ = model.Add(intermediate == product_d_pop)
    c_prod_min_d_population.append(intermediate)

In [10]:
# Setup minimization
objective = model.NewIntVar(0, MAX_DISTANCE*MAX_POPULATION, 'objective')

model.AddMaxEquality(objective, c_prod_min_d_population)
_ = model.Minimize(objective)

In [18]:
model.Validate()

''

### Step 3 : Constrains add

---

* **Modeling constrains**:
    * **Constrain 1**: Each job must be assigned only once
* **Input constrains**:
    * **Constrain 2**: Predefined constrains

**Constrain 1**: Each job must be assigned only once

**Constrain 2**: Each job must be assigned only once

### 4. Solving optimization

### 5. Results visualization